In [1]:
!pip install tensorflow==2.2.0
!python --version
import tensorflow
import tensorflow_hub
print(tensorflow.__version__)
print(tensorflow_hub.__version__)

     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 3.0MB 51.6MB/s 
     |████████████████████████████████| 460kB 57.3MB/s 
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0
Python 3.6.9
2.2.0
0.10.0


In [2]:
!pip3 install sentencepiece
!pip3 install tf-sentencepiece

     |████████████████████████████████| 1.1MB 18.0MB/s 
     |████████████████████████████████| 2.1MB 17.9MB/s 


In [9]:
import pandas as pd

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
german_sentences = ["Hund", "Welpen sind nett.", "Ich genieße lange Spaziergänge am Strand entlang mit meinem Hund."]

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module('https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1')
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

# Compute embeddings.
en_result = session.run(embedded_text, feed_dict={text_input: [english_sentences[0]]})
de_result = session.run(embedded_text, feed_dict={text_input: [german_sentences[0]]})

# Compute similarity. Higher score indicates greater similarity.
similarity_score = np.dot(np.squeeze(en_result), np.squeeze(de_result))


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [14]:
tuples = []

for english_sentence, german_sentence in zip(english_sentences, german_sentences):
  # Compute embeddings.
  en_result = session.run(embedded_text, feed_dict={text_input: [english_sentence]})
  de_result = session.run(embedded_text, feed_dict={text_input: [german_sentence]})

  # Compute similarity. Higher score indicates greater similarity.
  similarity_score = np.dot(np.squeeze(en_result), np.squeeze(de_result))

  # tuples.append((english_sentence, german_sentence, similarity_score, en_result, de_result))
  tuples.append((english_sentence, german_sentence, similarity_score))

# pd.DataFrame(tuples, columns = ['english_sentence', 'german_sentence', 'similarity_score', 'en_embedding', 'de_embedding'])
pd.set_option('display.max_colwidth', None)
pd.DataFrame(tuples, columns = ['english_sentence', 'german_sentence', 'similarity_score'])

,english_sentence,german_sentence,similarity_score
0,dog,Hund,0.900837
1,Puppies are nice.,Welpen sind nett.,0.909202
2,I enjoy taking long walks along the beach with my dog.,Ich genieße lange Spaziergänge am Strand entlang mit meinem Hund.,0.939364


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np

en_embedding_file = '/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/en_embeddings.npy'
de_embedding_file = '/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/de_embeddings.npy'

en_embeddings = np.load(en_embedding_file)
de_embeddings = np.load(de_embedding_file)
# embeddings = np.concatenate((en_embeddings, de_embeddings), axis = 0) 

# print('Shape of embeddings:', embeddings.shape)

In [3]:
%cd '/content/drive/My Drive/Colab Notebooks/media-agenda/'
import json

path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_dict.json'
with open(path, 'r') as f:
    sentences = json.load(f)
print(len(sentences))
print(sentences[0])

/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda
328091
{'0': "Customers at McDonald's restaurants in New England are about to get something a little different when they order coffee."}


In [15]:
import pandas as pd

result = []
count = 5

for en_index, en_embedding in enumerate(en_embeddings):
  tuples = []
  for de_index, de_embedding in enumerate(de_embeddings):
  result.append(pd.DataFrame(tuples, columns = ['en_index', 'de_index', 'similarity_score']).sort_values('similarity_score', ascending = False).head(count).to_numpy())
    similarity_score = np.dot(np.squeeze(en_embedding), np.squeeze(de_embedding))
    de_index = en_embeddings.shape[0] + de_index
    tuples.append((en_index, de_index, similarity_score))
    
  if en_index%10000 == 0:
    print('Now...', en_index)

result
# tuples = np.asarray(tuples)

# pd.set_option('display.max_colwidth', None)


Now... 0
Now... 10000
Now... 20000
Now... 30000


KeyboardInterrupt: ignored

In [17]:
np.save('cs_result', result)

In [18]:
result

[array([[0.0000000e+00, 2.1553500e+05, 6.4130795e-01],
        [0.0000000e+00, 2.8467600e+05, 6.4130795e-01],
        [0.0000000e+00, 2.1550900e+05, 6.4130795e-01],
        [0.0000000e+00, 2.1557200e+05, 6.4130795e-01],
        [0.0000000e+00, 3.1152300e+05, 6.4130795e-01]]),
 array([[1.00000000e+00, 1.29765000e+05, 7.22950697e-01],
        [1.00000000e+00, 3.19149000e+05, 7.16172040e-01],
        [1.00000000e+00, 3.19156000e+05, 7.02511787e-01],
        [1.00000000e+00, 3.19129000e+05, 6.97887182e-01],
        [1.00000000e+00, 3.19155000e+05, 6.73288465e-01]]),
 array([[2.00000000e+00, 1.30895000e+05, 7.81284153e-01],
        [2.00000000e+00, 3.27884000e+05, 7.79978156e-01],
        [2.00000000e+00, 2.22984000e+05, 7.77856708e-01],
        [2.00000000e+00, 1.48632000e+05, 7.74312258e-01],
        [2.00000000e+00, 3.25526000e+05, 7.70345807e-01]]),
 array([[3.00000000e+00, 1.29280000e+05, 7.74563909e-01],
        [3.00000000e+00, 1.93673000e+05, 7.05260992e-01],
        [3.00000000e+00